In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re


def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zğüşöçıİĞÜŞÖÇ\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

data = pd.read_csv('anlamsal_arama_data.csv', sep='|')

data['text'] = data['text'].apply(preprocess_text)

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['text'])

def semantic_search(query, tfidf_matrix, vectorizer, top_n=5):
    query_vector = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-top_n-1:-1]
    return related_docs_indices

# Load queries from the CSV file
queries_data = pd.read_csv('task2_yarismaci_test_data.csv', sep='|')
queries = queries_data['sorgu'].tolist()

results = []

for query in queries:
    indices = semantic_search(query, tfidf_matrix, vectorizer)
    top_results = data.iloc[indices]['title'].tolist()
    results.append([query] + top_results)

# Create a DataFrame
results_df = pd.DataFrame(results, columns=['query', '1st', '2nd', '3rd', '4th', '5th'])

# Write DataFrame to CSV file
results_df.to_csv('2result.csv', sep='|', index=False)

print("Results written to 2result.csv.")

# Print queries along with guesses
for query, guesses in zip(queries, results):
    print("Query:", query)
    print("Guesses:", guesses)
    print()

print("Results printed.")


Results written to 2result.csv.
Query: Doğu Anadolu Bölgesi'nde tarım alanlarının kapladığı yüzölçümü ne kadardır?
Guesses: ["Doğu Anadolu Bölgesi'nde tarım alanlarının kapladığı yüzölçümü ne kadardır?", 'Tarım', 'Doğu Anadolu Bölgesi', 'Garça', 'Güneydoğu Anadolu Bölgesi', 'Tarım ve Orman Bakanlığı']

Query: Bu metinde hangi Mısır tanrılarından bahsedilmektedir?
Guesses: ['Bu metinde hangi Mısır tanrılarından bahsedilmektedir?', 'Antik Mısır', 'Aşağı Mısır', 'Mısır Kupası', 'Mısır Silahlı Kuvvetleri Yüksek Konseyi', '1952 Mısır Devrimi']

Query: Terry Gilliam hangi filmin çekimlerini tamamlayamadan askıya almıştır?
Guesses: ['Terry Gilliam hangi filmin çekimlerini tamamlayamadan askıya almıştır?', 'Terry Gilliam', 'It Comes at Night', 'Ben-Hur (film, 2016)', 'Clarence Avant', '100 Years (film)']

Query: Avrupa Jeologlar Federasyonu, bugün 70.000'den fazla jeologu temsil eden bir organizasyon mudur?
Guesses: ["Avrupa Jeologlar Federasyonu, bugün 70.000'den fazla jeologu temsil eden bir